In [ ]:
def sidechain2(foreground_mix, background_mix, fs, threshold, ratio, attack, release):
    output = np.zeros(len(foreground_mix))
    
    attack_ramp = np.linspace(1, ratio, int(fs*attack))
    release_ramp = np.linspace(ratio, 1, int(fs*release))
    current_compression_rate = 1
    attacking = False
    releasing = False
    holding = False
    i = 0
    while i < len(output):
        if(abs(foreground_mix[i]) > threshold):
            # we were not compressing, we are attacking:
            attacking = True
            
            for t in np.arange(min(len(attack_ramp), len(output)-i)):
                compression_rate_index = 0
                # we compress with the current compression ratio until attack time is done
                if(abs(foreground_mix[i]) > threshold):
                    if(holding):
                        # we were holding a value compression during the attacktime
                        # but we cross the threadhold again, attack from the beginning
                        # but maintainig the compression_rate_index
                        t = 0
                        holding = False
                    # we are still over the threshold (or went back over the th), lest compress using the next compression rate
                    current_compression_rate = attack_ramp[compression_rate_index]
                    if(background_mix[i] > 0):
                        
                        current_compression_value = abs(foreground_mix[i] - threshold)/current_compression_rate
                        output[i] = foreground_mix[i] + background_mix[i]
                    else:
                        current_compression_value = abs(foreground_mix[i] - threshold)/current_compression_rate
                        output[i] = foreground_mix[i] + background_mix[i]
                        ######## ???????????????
                    compression_rate_index = min(compression_rate_index + 1, len(attack_ramp))
                else:
                    # we are still on attack time, but not over the threshold anymore
                    # we compress using the last compression value
                    holding = True
                    output[i] = foreground_mix[i] + (background_mix[i] - current_compression_value)
                i += 1
            attacking = False
            # now we release

            
#             for t in np.arange(min(len(release_ramp), len(output)-i)):
#                 compression_rate_index = 0
#                 if(abs(foreground_mix[i]) <= threshold):
# #                     if(holding):
# #                         # we were holding a value compression during the attacktime
# #                         # but we cross the threadhold again, attack from the beginning
# #                         # but maintainig the compression_rate_index
# #                         t = 0
# #                         holding = False
# #                     # we are still over the threshold (or went back over the th), lest compress using the next compression rate
#                     current_compression_rate = release_ramp[compression_rate_index]
#                     current_compression_value = (background_mix[i] - threshold)/current_compression_rate
#                     output[i] = foreground_mix[i] + (background_mix[i] - current_compression_value)
#                     compression_rate_index = min(compression_rate_index + 1, len(release_ramp))
#                     i += 1
#                 else:
#                     # we are still on attack time, but not over the threshold anymore
#                     # we compress using the last compression value
#                     break
        else:
            output[i] = foreground_mix[i] + background_mix[i]
            i += 1
    return output